In [ ]:
try:
    from google.colab import drive
    NOTEBOOK = 'colab'
except:
    import os    
    if list(os.walk('/kaggle/input')):            
        NOTEBOOK = 'kaggle'
    else:
        NOTEBOOK = 'home'
        
if NOTEBOOK == 'colab':
    drive.mount("/content/gdrive", force_remount=True)

import tensorflow as tf
# tf.debugging.set_log_device_placement(True)
# tf.config.set_visible_devices([], 'GPU') 
# tf.executing_eagerly() 

print(NOTEBOOK)




In [ ]:
if NOTEBOOK == 'colab':
    !pip install -q dm-sonnet
    !pip install -q gdown
    import gdown    
    url = 'https://drive.google.com/uc?id=1Si-Fz-38xITBOvYOoPTn_xLtOHVwqFQO'
    output = 'GOLD_XYZ.hdf5'
    gdown.download(url, output, quiet=False)
    !df -h
    !ls -l
    !grep Model: /proc/driver/nvidia/gpus/*/information | awk '{$1="";print$0}'
    hdf5_path = '/content/gdrive/My Drive/data/GOLD_XYZ.hdf5'
elif NOTEBOOK == 'kaggle':
    !pip install -q dm-sonnet
    import os
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            hdf5_path = os.path.join(dirname, filename)
else:          
    hdf5_path = '/media/bognev/CE50072F50071DB9/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5'
print(hdf5_path)


In [ ]:
# import tensorflow as tf
# tf.debugging.set_log_device_placement(True)
# import h5py
import numpy as np
import matplotlib.pyplot as plt
import random
import tqdm
import time
import h5py
import sonnet as snt
from tqdm import tqdm

print("TensorFlow version: {}".format(tf.__version__))
print("    Sonnet version: {}".format(snt.__version__))
print("    Numpy  version: {}".format(np.__version__))
from tensorboard import version; print("TensorBoard version: {}".format(version.VERSION))
# tf.config.list_physical_devices('GPU')

In [ ]:
class ConvMaxPool(snt.Module):
    def __init__(self, name=None):
        super(ConvMaxPool, self).__init__(name=name)
        self.conv = snt.Conv1D(output_channels=64, kernel_shape=64, stride=1, rate=1,
                               padding="SAME", with_bias=True,
                               data_format="NWC", name="conv_max_pool"
                               )

    def __call__(self, inputs):
        features = self.conv(inputs)
        outputs = tf.nn.max_pool1d(features, ksize=2, strides=2, padding="VALID", 
                                   data_format='NWC', name="pool") #64×1024
        return outputs


class CNN(snt.Module):
    def __init__(self, epsilon, name=None):
        super(CNN, self).__init__(name=name)
        self.layers = [
            ConvMaxPool(name="conv"+str(ii))
            for ii in range(6)
        ]
        self.conv_in = snt.Conv1D(output_channels=64, kernel_shape=2, stride=1, rate=1,
                       padding="SAME", with_bias=True,
                       data_format="NWC", name="conv_input"
                       )

        self.linear = snt.Linear(output_size=128, with_bias=True)
        self.linear1 = snt.Linear(output_size=128, with_bias=True)
        self.linear2 = snt.Linear(output_size=24, with_bias=True)        
        self.decay_lr = 0
        self.lr=2e-4


    # @snt.once
    def __initialize(self, weight):
        pass

    def __call__(self, inputs):
        features = self.conv_in(inputs) #2×1024
        features = tf.nn.max_pool1d(features, ksize=2, strides=2, padding="VALID", 
                                    data_format='NWC', name="pool_input") 
#         print(features)
        for layer in self.layers:
            features = layer(features)    
        features = snt.flatten(features)

        features = self.linear(features)
        features = tf.nn.relu(features)
        features = self.linear1(features)
        features = tf.nn.relu(features)

        output = self.linear2(features)        
#         output = tf.nn.softmax(features)
        return output


In [ ]:
class CNNOptimizer(snt.Module):
    def __init__(self, name, cnn, lr=1e-3, num_epochs=100, decay_lr_start_epoch=10, decay_lr=False):
        super(CNNOptimizer, self).__init__(name=name)
        self.cnn = cnn
        self.init_lr = lr
        self.lr = tf.Variable(lr, trainable=False, name='learning rate', dtype=tf.float32)
        self.cnn_opt = snt.optimizers.Adam(learning_rate=self.lr, beta1=0.)
        self.decay_lr_start_epoch = tf.constant(decay_lr_start_epoch, dtype=tf.int32)
        self.decay_lr = tf.constant(decay_lr, dtype=tf.float32)
        self.num_epochs = tf.constant(num_epochs, dtype=tf.int32)

    def _gen_lr_mult(self, epoch):
          # Linear decay to 0.
        decay_epoch = tf.cast(epoch - self.decay_lr_start_epoch, tf.float32)
        if decay_epoch < tf.constant(0, dtype=tf.float32):
            return tf.constant(1., dtype=tf.float32)
        num_decay_epochs = tf.cast(self.num_epochs - self.decay_lr_start_epoch, dtype=tf.float32)
        return (num_decay_epochs - decay_epoch) / num_decay_epochs        
      
    def step(self, batch, labels, epoch):
        with tf.GradientTape() as tape:
            logits = self.cnn(batch)    
            loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
            loss = tf.reduce_mean(loss)
        params = radio_model.trainable_variables
        grads = tape.gradient(loss, params)                
        lr_mult = self._gen_lr_mult(epoch)
        if self.decay_lr:
            self.lr.assign(self.init_lr * lr_mult)
        self.cnn_opt.apply(grads, params)
        return loss
    




In [ ]:
radio_model = CNN(epsilon=1e-8, name="CNN")

# print(radio_model)
# print(radio_opt)
# radio_opt._gen_lr_mult
# radio_opt.step


In [ ]:
class radioML:
    def __init__(self, file, batch_size):
        self.file = file
        self.batch_size = batch_size

    def __call__(self):
        with h5py.File(self.file, 'r') as f:
            datasize = len(f['X'])
            while True:
                idxs = random.sample(range(datasize), self.batch_size)
                yield (f['X'][sorted(idxs)], f['Y'][sorted(idxs)])
                # f.close()

def radioML_map(data_batch, data_label):
#     print(data_batch.shape)
    data_batch = tf.reshape(data_batch, [1024, 2])
#     mean = tf.math.reduce_mean(data_batch)
#     std = tf.math.reduce_std(data_batch)
#     data_batch = (data_batch - mean)/std
#     print(std, mean)
#     print(type(std))
    return (data_batch, data_label)

In [ ]:
radio_plot = tf.data.Dataset.from_generator(radioML(hdf5_path, 1), 
                output_types=(tf.float32, tf.float32),
                output_shapes=(tf.TensorShape([1, 1024, 2]), tf.TensorShape([1, 24]))) #2555904, 1024, 2]

radio_plot = radio_plot.take(1)
radio_plot = radio_plot.map(radioML_map)
for (batch, labels) in radio_plot:    
    print(batch.shape)
    fig1 = plt.figure()
    plt.plot(batch[:,0])
    plt.xlabel('Batch #')
    plt.ylabel('Loss [entropy]')
    fig2 = plt.figure()
    plt.plot(batch[:,1])
    plt.xlabel('Batch #')
    plt.ylabel('Loss [entropy]')
    plt.show()

In [ ]:
%reload_ext tensorboard
# %load_ext tensorboard
# import os
from datetime import datetime as dt
# Clear any logs from previous runs
!rm -rf ./logs/
log_dir = "logs/scalars/" + dt.now().strftime("%Y%m%d-%H%M%S")
writer = tf.summary.create_file_writer(log_dir)


writer.set_as_default()
%tensorboard --logdir logs/scalars

In [ ]:

batch_size = 64
num_batches = 64
num_epochs = 100

radio_opt = CNNOptimizer(name="Optimizer", cnn=radio_model, lr=0.001, \
                         num_epochs=num_epochs, decay_lr_start_epoch=5, decay_lr=True)

t = tqdm(range(num_batches * num_epochs),
                                unit='sig', unit_scale=batch_size,
                                position=0)


radio_data = tf.data.Dataset.from_generator(radioML(hdf5_path, 1), 
                output_types=(tf.float32, tf.float32),
                output_shapes=(tf.TensorShape([1, 1024, 2]), tf.TensorShape([1, 24]))) #2555904, 1024, 2]

loss_history = []
loss = 0
step_num=0
radio_data = radio_data.take(batch_size*num_batches)
radio_data = radio_data.map(radioML_map)
radio_data = radio_data.batch(batch_size)
radio_data = radio_data.prefetch(tf.data.experimental.AUTOTUNE)
radio_data = radio_data.repeat(num_epochs)
radio_data = radio_data.cache()

# step = tf.function(radio_opt.step)

start_time = time.perf_counter()
for step_num,(batch, labels) in enumerate(radio_data):      
    epoch = tf.constant(1+int(step_num / (num_batches)))
    loss = radio_opt.step(batch, labels, epoch)
    step_num+=1
    with writer.as_default():
        tf.summary.scalar('training loss', loss, step=step_num)
        tf.summary.scalar('learning rate', radio_opt.lr, step=step_num)    
    t.update(1)
    if step_num % num_batches == 0:
        t.write('Epoch = {}/{} (lr_mult = {:0.05f}, loss = {}) done.'.format(
            epoch.numpy(), num_epochs, radio_opt.lr.numpy(), loss.numpy()))
tf.print("Execution time:", time.perf_counter() - start_time)   
t.close()
# tf.print("batch.shape: ", i, batch.shape)  
# tf.print(loss_history)                        

In [ ]:
tf.math.is_nan(loss_history)
loss_history = loss_history
fig2 = plt.figure()
plt.plot(loss_history)
plt.xlabel('Batch #')
plt.ylabel('Loss [entropy]')
plt.show()
loss_history

In [ ]:
print(snt.format_variables(radio_model.variables))